In [1]:
import pandas as pd
from sshtunnel import SSHTunnelForwarder
import pymongo

In [2]:
# 連到本地端 mongo
client = pymongo.MongoClient(host='mongodb://localhost:27017/', port=27017)

In [3]:
# 沒有跳出 timeouterror 代表有連線成功
client.server_info()

{'version': '4.2.6',
 'gitVersion': '20364840b8f1af16917e4c23c1b5f5efd8b352f8',
 'modules': [],
 'allocator': 'system',
 'javascriptEngine': 'mozjs',
 'sysInfo': 'deprecated',
 'versionArray': [4, 2, 6, 0],
 'openssl': {'running': 'Apple Secure Transport'},
 'buildEnvironment': {'distmod': '',
  'distarch': 'x86_64',
  'cc': '/Applications/Xcode10.2.0.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/clang: Apple LLVM version 10.0.1 (clang-1001.0.46.3)',
  'ccflags': '-isysroot /Applications/Xcode10.2.0.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX10.14.sdk -mmacosx-version-min=10.12 -target darwin16.0.0 -arch x86_64 -fno-omit-frame-pointer -fno-strict-aliasing -ggdb -pthread -Wall -Wsign-compare -Wno-unknown-pragmas -Winvalid-pch -Werror -O2 -Wno-unused-local-typedefs -Wno-unused-function -Wno-unused-private-field -Wno-deprecated-declarations -Wno-tautological-constant-out-of-range-compare -Wno-tautological-constant-compare -Wno-tautological-u

In [4]:
# 擷取 database
database = client['test']

In [5]:
# 擷取 collections(table)
collection = database['col_test']

In [6]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test'), 'col_test')

In [7]:
# truncate collection(table)
collection.remove()

# 插入資料
doc1 = {"name":"Lucy", "sex":"female","job":"nurse", "salary":50}
doc2 = {"name":"Jerry", "sex":"male","job":"doctor", "salary":100}
doc3 = {"name":"Wendell", "sex":"male","job":"Engineer", "salary":200}
doc4 = {"name":"Damon", "sex":"male","job":"Engineer", "salary":150}
doc5 = {"name":"Anne", "sex":"female","job":"art", "salary":120}
collection.insert([doc1, doc2, doc3, doc4, doc5])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  # Remove the CWD from sys.path while we load stuff.


[ObjectId('5ec2571ca4be69b622701185'),
 ObjectId('5ec2571ca4be69b622701186'),
 ObjectId('5ec2571ca4be69b622701187'),
 ObjectId('5ec2571ca4be69b622701188'),
 ObjectId('5ec2571ca4be69b622701189')]

In [8]:
# 這個 find() 只會返還一個 cursor, 必須用 for loop 把資料都取出來。g
data = collection.find() # 會返回一個 cursor

In [9]:
for d in data:
    print(d)

{'_id': ObjectId('5ec2571ca4be69b622701185'), 'name': 'Lucy', 'sex': 'female', 'job': 'nurse', 'salary': 50}
{'_id': ObjectId('5ec2571ca4be69b622701186'), 'name': 'Jerry', 'sex': 'male', 'job': 'doctor', 'salary': 100}
{'_id': ObjectId('5ec2571ca4be69b622701187'), 'name': 'Wendell', 'sex': 'male', 'job': 'Engineer', 'salary': 200}
{'_id': ObjectId('5ec2571ca4be69b622701188'), 'name': 'Damon', 'sex': 'male', 'job': 'Engineer', 'salary': 150}
{'_id': ObjectId('5ec2571ca4be69b622701189'), 'name': 'Anne', 'sex': 'female', 'job': 'art', 'salary': 120}


In [10]:
# 查找特定值
data = collection.find({"name":"Jerry"})
for d in data:
    print(d)

{'_id': ObjectId('5ec2571ca4be69b622701186'), 'name': 'Jerry', 'sex': 'male', 'job': 'doctor', 'salary': 100}


In [11]:
# select * from collection where sex=female and salary>100
data = collection.find({'$and':[
            {'sex': 'female'},
            {'salary': {'$gt':100}} # gt for greater than; gte for greater than & equal to
            ]
        })
for d in data:
    print(d)

{'_id': ObjectId('5ec2571ca4be69b622701189'), 'name': 'Anne', 'sex': 'female', 'job': 'art', 'salary': 120}


In [12]:
# 限制查詢個數
data = collection.find({"sex":"male"}).limit(2)
for d in data:
    print(d)

{'_id': ObjectId('5ec2571ca4be69b622701186'), 'name': 'Jerry', 'sex': 'male', 'job': 'doctor', 'salary': 100}
{'_id': ObjectId('5ec2571ca4be69b622701187'), 'name': 'Wendell', 'sex': 'male', 'job': 'Engineer', 'salary': 200}


In [13]:
# 排序、反向排序
data = collection.find().sort("salary") # 預設為正向排序
data = collection.find().sort("salary", pymongo.ASCENDING)
data = collection.find().sort("salary", pymongo.DESCENDING)
for d in data:
    print(d)

{'_id': ObjectId('5ec2571ca4be69b622701187'), 'name': 'Wendell', 'sex': 'male', 'job': 'Engineer', 'salary': 200}
{'_id': ObjectId('5ec2571ca4be69b622701188'), 'name': 'Damon', 'sex': 'male', 'job': 'Engineer', 'salary': 150}
{'_id': ObjectId('5ec2571ca4be69b622701189'), 'name': 'Anne', 'sex': 'female', 'job': 'art', 'salary': 120}
{'_id': ObjectId('5ec2571ca4be69b622701186'), 'name': 'Jerry', 'sex': 'male', 'job': 'doctor', 'salary': 100}
{'_id': ObjectId('5ec2571ca4be69b622701185'), 'name': 'Lucy', 'sex': 'female', 'job': 'nurse', 'salary': 50}


In [14]:
# 直接把 cursor 結果變成一個 list or dataframe
data = collection.find()
df = list(data)
pd.DataFrame(df)

,_id,name,sex,job,salary
0,5ec2571ca4be69b622701185,Lucy,female,nurse,50
1,5ec2571ca4be69b622701186,Jerry,male,doctor,100
2,5ec2571ca4be69b622701187,Wendell,male,Engineer,200
3,5ec2571ca4be69b622701188,Damon,male,Engineer,150
4,5ec2571ca4be69b622701189,Anne,female,art,120


***

## Access in im_internal text env

In [16]:
# jump.svc.imvideo.tv # 10.25.70.69
MONGO_HOST = "127.0.0.1"
# MONGO_DB = "DATABASE_NAME"
MONGO_USER = "im_data_user"
MONGO_PASS = "a945dcc8510a19883a367be9b3f41c2fa81e5f9b"

server = SSHTunnelForwarder(
    ("jump.svc.imvideo.tv",22),
    ssh_username="jerry_chen",
#     ssh_password=MONGO_PASS,
    ssh_pkey='/Users/user/.ssh/id_rsa_mongo',
    remote_bind_address=('mongos-auth.test.intl.imvideo.tv', 27017)
)

server.start()

client = pymongo.MongoClient('127.0.0.1', server.local_bind_port) # server.local_bind_port is assigned local port
# db = client[MONGO_DB]
# print(db.collection_names())

# server.stop()

In [ ]:
# 沒有跳出 timeouterror 代表有連線成功
client.server_info()

In [ ]:
# im_internal 這個 database 需要使用者、密碼做 auth
client['im_internal'].authenticate("im_data_user", "a945dcc8510a19883a367be9b3f41c2fa81e5f9b", mechanism='SCRAM-SHA-1')

In [ ]:
db_im_internal = client['im_internal']

In [ ]:
col_recording_like = db_im_internal['recording_like']

In [ ]:
docs = col_recording_like.find().limit(5)

In [ ]:
docs = col_recording_like.find()
df = pd.DataFrame(list(docs))

In [ ]:
df